In [2]:
from evaluation.mol_structure import list_of_smiles_to_nx_graphs
# Generate graphs for demonstration purposes
import utils.graph_generators as gen
import torch
import dgl
import networkx as nx
from evaluation.moses.metrics import get_all_metrics
from evaluation.evaluator import Evaluator
from evaluation.gin_evaluation import load_feature_extractor, MMDEvaluation
import copy


In [1]:
import numpy as np

In [3]:
a=gen.load_zinc(n_samples=10)
grids=a
lobsters=a
#print(len(grids[0].ndata['attr']))
#print(grids[0].ndata['attr']  , grids[0].ndata['feat'])

In [4]:


lobsters_nx = [ dgl.to_networkx(g, node_attrs=['attr','feat'], edge_attrs=['attr','feat'])  for g in a]
grids_nx = [dgl.to_networkx(g, node_attrs=['attr','feat'], edge_attrs=['attr','feat']) for g in a]

#device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

device=torch.device('cpu')

grids = [ dgl.from_networkx(g,node_attrs=['feat'], edge_attrs=['feat']).to(device) for g in grids_nx] # Convert graphs to DGL from NetworkX
lobsters = [ dgl.from_networkx(g,node_attrs=['feat'], edge_attrs=['feat']).to(device) for g in lobsters_nx] # Convert graphs to DGL from NetworkX

In [5]:

# Compute all GNN-based metrics at once
evaluator = Evaluator(device=device)
all_gnn=evaluator.evaluate_all(generated_dataset=grids, reference_dataset=lobsters)
print(all_gnn)
 #Alternatively, compute a single GNN-based metric. See evaluation/gin_evaluation.py for other metrics.
# Can tweak GIN hyperparameters, however defaults are set to our recommendations
gin = load_feature_extractor(device=device)
# Can tweak hyperparameters of MMD RBF, however defaults are set to our recommendations
mmd_eval = MMDEvaluation(gin)
result, time = mmd_eval.evaluate(generated_dataset=grids, reference_dataset=lobsters)
print('The single gin  metric recommended by the https://github.com/uoguelph-mlrg/GGM-metrics/search?q=prdcEvaluation: {}, time to compute: {:.3f}s'.format(result, time))

# Evaluation incorporating custom node/edge features.
# If no node features are provided, the default is degree features
# expressed as an integer.

node_feat_loc = 'feat'
edge_feat_loc = 'feat'
node_feat_dim = 10
edge_feat_dim = 5

# Create random node/edge features
for g in grids:
    num_nodes = g.number_of_nodes()
    g.ndata[node_feat_loc] = torch.randn(num_nodes, node_feat_dim).to(g.device)
    num_edges = g.number_of_edges()
    g.edata[edge_feat_loc] = torch.randn(num_edges, edge_feat_dim).to(g.device)
    
for g in lobsters:
    num_nodes = g.number_of_nodes()
    g.ndata[node_feat_loc] = torch.randn(num_nodes, node_feat_dim).to(g.device)
    num_edges = g.number_of_edges()
    g.edata[edge_feat_loc] = torch.randn(num_edges, edge_feat_dim).to(g.device)

# Load GIN
gin = load_feature_extractor(
    device=device, input_dim=node_feat_dim, edge_feat_dim=edge_feat_dim,
    node_feat_loc=node_feat_loc, edge_feat_loc=edge_feat_loc)

mmd_eval = MMDEvaluation(gin)
result, time = mmd_eval.evaluate(generated_dataset=lobsters, reference_dataset=grids)
print('result: {}, time to compute: {:.3f}s'.format(result, time))

# To quickly demonstrate sensitivity to node/edge features, let's create
# a copy of the grid graphs and slightly alter the node/edge feat distributions

result, time = mmd_eval.evaluate(generated_dataset=grids, reference_dataset=grids)
print('MMD RBF for grids with itself: {}'.format(result['mmd_rbf']))

grids_copy = [copy.deepcopy(g) for g in grids]
for g in grids_copy:
    g.ndata[node_feat_loc] *= 1.15
    g.edata[edge_feat_loc] *= 1.15
    
result, time = mmd_eval.evaluate(reference_dataset=grids, generated_dataset=grids_copy)
print('MMD RBF after altering node/edge feat. distributions: {}'.format(result['mmd_rbf']))


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
{'activations_time': 0.03400278091430664, 'fid': -7.276851491155867e-06, 'fid_time': 0.060003042221069336, 'kid': -0.6868758, 'kid_time': 10.257941007614136, 'precision': 1.0, 'recall': 1.0, 'f1_pr': 1.00001, 'precision_time': 0.0614469051361084, 'recall_time': 0.0614469051361084, 'f1_pr_time': 0.0614469051361084, 'density': 1.2000000000000002, 'coverage': 1.0, 'f1_dc': 1.090919173552968, 'density_time': 0.0500030517578125, 'coverage_time': 0.0500030517578125, 'f1_dc_time': 0.0500030517578125, 'mmd_rbf': 0, 'mmd_rbf_time': 0.06000518798828125, 'mmd_linear': 0.0, 'mmd_linear_time': 0.03400278091430664}
The single gin  metric recommended by the https://github.com/uoguelph-mlrg/GGM-metrics/search?q=prdcEvaluation: {'mmd_rbf': 0}, time to compute: 0.050s
re

In [6]:
evaluator = Evaluator(device=device, metrics_type=['gin','structure', 'molecular'],feature_extractor='mmd-structure',statistic='all' )
evaluator.evaluate_all(generated_dataset=grids, reference_dataset=lobsters)

c:\Users\cs843\Documents\PhD\2023\Proposal\GGM-metrics\evaluation\graph_structure_evaluation.py:128: RuntimeWarning: Mean of empty slice.
  mmd = K_GG.mean() + K_RR.mean() - (2 * K_GR.mean())
c:\Users\cs843\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
<class 'networkx.utils.decorators.argmap'> compilation 8:4: FutureWarning: normalized_laplacian_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.


{'nspdk_mmd': 0.0,
 'nspdk_mmd_time': 0.18408966064453125,
 'wl_mmd': 0.0,
 'wl_mmd_time': 0.021998882293701172,
 'degree_mmd': 0,
 'degree_mmd_time': 52.47593832015991,
 'clustering_mmd': 0,
 'clustering_mmd_time': 53.26442575454712,
 'orbits_mmd': 0,
 'orbits_mmd_time': 0.07153749465942383,
 'spectral_mmd': 0,
 'spectral_mmd_time': 35.36649465560913}

In [7]:
# To quickly demonstrate sensitivity to node/edge features, let's create
# a copy of the grid graphs and slightly alter the node/edge feat distributions

result, time = mmd_eval.evaluate(generated_dataset=grids, reference_dataset=grids)
print('MMD RBF for grids with itself: {}'.format(result['mmd_rbf']))

grids_copy = [copy.deepcopy(g) for g in grids]
for g in grids_copy:
    g.ndata[node_feat_loc] *= 5
    g.edata[edge_feat_loc] *= 5
    
result, time = mmd_eval.evaluate(reference_dataset=grids, generated_dataset=grids_copy)
print('MMD RBF after altering node/edge feat. distributions: {}'.format(result['mmd_rbf']))

MMD RBF for grids with itself: 0
MMD RBF after altering node/edge feat. distributions: 1.0981200635433197


In [ ]:
#What sort of preprocessing  you need to make to be able to execute the metrics from moses

#https://github.com/graphdeeplearning/benchmarking-gnns/issues/42
#https://stackoverflow.com/questions/51195392/smiles-from-graph


In [8]:
lobsters = [ dgl.to_networkx(g, node_attrs=['attr','feat'], edge_attrs=['attr','feat'])  for g in a]
grids = [dgl.to_networkx(g, node_attrs=['attr','feat'], edge_attrs=['attr','feat']) for g in a]

In [158]:
discrete_node_label_name='attr' #leave blank if this does not exist, default: 'label'
 #leave blank if this does not exist, default: 'label'
continuous_node_label_name='feat'  #leave blank if this does not exist default: 'attr'
discrete_edge_label_name='attr'  #leave blank if this does not exist  ,default: 'label'
continuous_edge_label_name='feat' #leave blank if this does not exist , default: 'attr'
#dicrete labels should be set to 'label'
#continous labels should be set to 'attr'
#'nn and 'structural need dgl graphs, need conversion
def preprocess(nx_dataset,discrete_node_label_name, continuous_node_label_name,discrete_edge_label_name,  continuous_edge_label_name):
    processed_dataset=[]
    for G in nx_dataset:
        dict=nx.get_node_attributes(G, discrete_node_label_name)
        nx.set_node_attributes(G, dict, 'label') 
        dict=nx.get_node_attributes(G, continuous_node_label_name)
        nx.set_node_attributes(G, dict, 'attr')
        dict=nx.get_edge_attributes(G, discrete_edge_label_name)
        nx.set_node_attributes(G, dict, 'label')
        dict=nx.get_edge_attributes(G, continuous_edge_label_name)
        nx.set_node_attributes(G, dict, 'attr')
        processed_dataset.append(G)
    return processed_dataset

tr=preprocess(nx_lobsters,discrete_node_label_name, continuous_node_label_name, discrete_edge_label_name,continuous_edge_label_name)
tr[0].nodes(data=True)

NodeDataView({0: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor(0)}, 1: {'attr': tensor(1), 'feat': tensor(1), 'label': tensor(1)}, 2: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor(0)}, 3: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor(0)}, 4: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor(0)}, 5: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor(0)}, 6: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor(0)}, 7: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor(0)}, 8: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor(0)}, 9: {'attr': tensor(1), 'feat': tensor(1), 'label': tensor(1)}, 10: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor(0)}, 11: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor(0)}, 12: {'attr': tensor(2), 'feat': tensor(2), 'label': tensor(2)}, 13: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor(0)}, 14: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor(0)}, 15: {'attr': tensor(2), 'feat': tens

In [174]:
lobsters_nx[0].nodes(data=True)

NodeDataView({0: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])}, 1: {'attr': tensor(1), 'feat': tensor(1), 'label': tensor([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])}, 2: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])}, 3: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])}, 4: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])}, 5: {'attr': tensor(0), 'feat': tensor(0), 'label': tensor([1., 0., 0., 0., 0

In [5]:
device=torch.device('cpu')
metrics_type=['nn', 'structural', 'molecular']
structural_statistic='nspdk'  #options ['WL' , 'nspdk', 'orbits, 
#'degree', 'clustering' ,'spectral']


reference_graphs=lobsters_nx
generated_graphs=grids_nx

discrete_node_label_name='attr' #leave it blank if this does not exist, default: 'label'
 #leave blank if this does not exist, default: 'label'
continuous_node_label_name='feat'  #leave it blank if this does not exist default: 'attr'
discrete_edge_label_name='attr'  #leave it blank if this does not exist  ,default: 'label'
continuous_edge_label_name='feat' #leave it blank if this does not exist , default: 'attr'
#dicrete labels should be set to 'label'
#continous labels should be set to 'attr'


def preprocess(nx_dataset,discrete_node_label_name, continuous_node_label_name,discrete_edge_label_name,  continuous_edge_label_name):
    processed_dataset=[]
    for G in nx_dataset:
        dict=nx.get_node_attributes(G, discrete_node_label_name)
        nx.set_node_attributes(G, dict, 'label') 
        dict=nx.get_node_attributes(G, continuous_node_label_name)
        nx.set_node_attributes(G, dict, 'attr')
        dict=nx.get_edge_attributes(G, discrete_edge_label_name)
        nx.set_node_attributes(G, dict, 'label')
        dict=nx.get_edge_attributes(G, continuous_edge_label_name)
        nx.set_node_attributes(G, dict, 'attr')
        processed_dataset.append(G)
    return processed_dataset


reference_graphs=preprocess(reference_graphs,discrete_node_label_name, continuous_node_label_name, discrete_edge_label_name,continuous_edge_label_name)
generated_graphs=preprocess(generated_graphs,discrete_node_label_name, continuous_node_label_name, discrete_edge_label_name,continuous_edge_label_name)
reference_graphs_dgl = [ dgl.from_networkx(g,node_attrs=[continuous_node_label_name], edge_attrs=[continuous_edge_label_name]).to(device) for g in reference_graphs] # Convert graphs to DGL from NetworkX
generated_graphs_dgl = [ dgl.from_networkx(g,node_attrs=[continuous_node_label_name], edge_attrs=[continuous_edge_label_name]).to(device) for g in generated_graphs] # Convert graphs to DGL from NetworkX
metrics={}
if  'nn' in metrics_type:
    assert continuous_node_label_name!='' ,'You need continuous features to be able to compure nn-based metrics'
    eval=Evaluator(feature_extractor ='gin',device='cpu')
    nn_metrics=eval.evaluate_all(generated_dataset=reference_graphs_dgl,reference_dataset=reference_graphs_dgl)
    metrics.update(nn_metrics)
        
if 'structural' in metrics_type  :
    eval=Evaluator(feature_extractor ='mmd-structure',device='cpu' ,statistic=structural_statistic)
    structural_metrics=eval.evaluate_all(generated_dataset=generated_graphs_dgl,reference_dataset=reference_graphs_dgl)
    metrics.update(structural_metrics)
    
if 'molecular'  in metrics_type:
    #@fuckit
    def func(reference_graphs, generated_graphs):
        reference_graphs_mol= [nx_to_mol(g,edge_label='feat', node_label='feat') for g in reference_graphs]
        generated_graphs_mol=[nx_to_mol(g,edge_label='feat', node_label='feat') for g in generated_graphs]
        print('s')
        reference_graphs_smiles=mol_to_smiles(reference_graphs_mol)
        generated_graphs_smiles=mol_to_smiles(generated_graphs_mol)

        mol_metrics=get_all_metrics(gen=generated_graphs_smiles,train=reference_graphs_smiles)
        metrics.update(mol_metrics)
    try: 
        func(reference_graphs,generated_graphs)
    except: 
        print('Cannot compute molecular metrics for this type of graphs. You might neeed  to manually change the definition of nx_to_mol:)\n')
        
    
print(metrics)

orthogonal

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.map_fn(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.map_fn(fn, elems))
s


c:\Users\cs843\Documents\PhD\2023\February\GGM-metrics\evaluation\moses\metrics\metrics.py:216: UserWarning: Can't compute unique@1000.gen contains only 10 molecules
  warnings.warn(
c:\Users\cs843\Documents\PhD\2023\February\GGM-metrics\evaluation\moses\metrics\metrics.py:216: UserWarning: Can't compute unique@10000.gen contains only 10 molecules
  warnings.warn(


{'activations_time': 0.022013425827026367, 'fid': -6.695371780551795e-06, 'fid_time': 0.030015945434570312, 'kid': -0.71907973, 'kid_time': 6.75724196434021, 'precision': 1.0, 'recall': 1.0, 'f1_pr': 1.00001, 'precision_time': 0.0653543472290039, 'recall_time': 0.0653543472290039, 'f1_pr_time': 0.0653543472290039, 'density': 1.2000000000000002, 'coverage': 1.0, 'f1_dc': 1.090919173552968, 'density_time': 0.04326772689819336, 'coverage_time': 0.04326772689819336, 'f1_dc_time': 0.04326772689819336, 'mmd_rbf': 0, 'mmd_rbf_time': 0.05101323127746582, 'mmd_linear': 0.0, 'mmd_linear_time': 0.022013425827026367, 'nspdk_mmd': 0.0, 'nspdk_mmd_time': 0.18700027465820312, 'valid': 1.0, 'unique@1000': 1.0, 'unique@10000': 1.0, 'FCD/Test': 46.72970354700773, 'SNN/Test': 0.49758105874061587, 'Frag/Test': 0.6078516005720516, 'Scaf/Test': 0.029266636262645807, 'FCD/TestSF': 47.06954094903616, 'SNN/TestSF': 0.4751725435256958, 'Frag/TestSF': -0.03829807449991973, 'Scaf/TestSF': 0.0, 'IntDiv': 0.7798648

In [272]:
eval=Evaluator(feature_extractor ='mmd-structure',statistic='WL')
eval.evaluate_all(generated_dataset=grids,reference_dataset=lobsters)

{'wl_mmd': 0.0, 'wl_mmd_time': 0.019949913024902344}

In [273]:
eval=Evaluator(feature_extractor ='mmd-structure',statistic='nspdk')
eval.evaluate_all(generated_dataset=grids,reference_dataset=lobsters)

{'nspdk_mmd': 0.0, 'nspdk_mmd_time': 0.164459228515625}

In [274]:
eval=Evaluator(feature_extractor ='mmd-structure',statistic='clustering')
eval.evaluate_all(generated_dataset=grids,reference_dataset=lobsters)

{'clustering_mmd': 0, 'clustering_mmd_time': 68.12237238883972}

In [275]:
eval=Evaluator(feature_extractor ='mmd-structure',statistic='degree')
eval.evaluate_all(generated_dataset=grids,reference_dataset=lobsters)

{'degree_mmd': 0, 'degree_mmd_time': 62.189956188201904}

In [4]:
eval=Evaluator(feature_extractor ='mmd-structure',statistic='clustering')
eval.evaluate_all(generated_dataset=grids,reference_dataset=lobsters)

{'clustering_mmd': 0, 'clustering_mmd_time': 55.546926975250244}

In [5]:

#beware the graph statistics metric it is very slow
eval=Evaluator(feature_extractor ='mmd-structure',statistic='all')
metrics=eval.evaluate_all(generated_dataset=grids[:5],reference_dataset=lobsters[5:])